In [2]:
from base64 import encode
import requests
from bs4 import BeautifulSoup
import csv
from itertools import zip_longest
import pandas as pd
import unicodedata
import re
from queue import Queue
from threading import Thread
from collections import deque

link = "https://https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/page/"


## scrap all booklets links

In [26]:
weekly_offers_links = []
for i in range(5):
    src = requests.get(link+str(i)).content
    soup = BeautifulSoup(src, "lxml")
    weekly_offers_links.extend(soup.select("a.more-link"))


In [27]:
for i in range(len(weekly_offers_links)):
    weekly_offers_links[i] = weekly_offers_links[i].attrs["href"]

In [28]:
pd.DataFrame({"weekly_offers_links":weekly_offers_links}).to_csv(r'data\tamimi\weekly_offers_links.csv')

In [46]:
soup.select("h1.post-title")

[]

## load all booklets links

In [9]:
weekly_offers_links= pd.read_csv(r'data\othaim\weekly_offers_links.csv')["weekly_offers_links"]

In [10]:
weekly_offers_links

0      https://www.alsoouq.com/saudi-arabia-electroni...
1      https://www.alsoouq.com/saudi-arabia-offers/of...
2      https://www.alsoouq.com/saudi-arabia-offers/of...
3      https://www.alsoouq.com/saudi-arabia-offers/of...
4      https://www.alsoouq.com/saudi-arabia-offers/of...
                             ...                        
145    https://www.alsoouq.com/saudi-arabia-offers/of...
146    https://www.alsoouq.com/saudi-arabia-offers/of...
147    https://www.alsoouq.com/saudi-arabia-offers/of...
148    https://www.alsoouq.com/saudi-arabia-electroni...
149    https://www.alsoouq.com/saudi-arabia-offers/of...
Name: weekly_offers_links, Length: 150, dtype: object

In [20]:
import os
from turtle import st


page_images_all = pd.DataFrame({"img_link":[],"image_page_number":[],"booklet_name":[],"booklet_link":[]})
for i in range(9):

    print("in weekly_offers_links: " + str(i))
    current_weekly_offers_link=weekly_offers_links[i]
    res=requests.get(current_weekly_offers_link)
    src=res.content
    soup = BeautifulSoup(src, "lxml")

# get the number of web pages of this weekly offers booklest 
    numOfWebPages=soup.select('.page-link a')
    if(numOfWebPages):
        numOfWebPages=numOfWebPages[len(numOfWebPages)-1].text
        numOfWebPages = int(numOfWebPages)
    else:
       numOfWebPages=1 
    
    booklet_name = soup.select("h1.post-title")[0].text
    
    page_images_booklet = pd.DataFrame({"img_link":[],"image_page_number":[],"booklet_name":[],"booklet_link":[]})
    for j in range(1,numOfWebPages+1):
        print("in numOfWebPages: "+ str(j))
        src = requests.get(current_weekly_offers_link+"/"+str(j)).content
        soup = BeautifulSoup(src, "lxml")
        img_links =soup.select('noscript img[style="border: 0px;"]')
        page_images = pd.DataFrame({"img_link":img_links,"image_page_number":j,"booklet_name":booklet_name,"booklet_link":current_weekly_offers_link})
        page_images["img_link"]= page_images["img_link"].apply(lambda x: x.attrs["src"])
        page_images_booklet=pd.concat([page_images_booklet,page_images])
    # os.mkdir(r'C:\Users\user\Desktop\web scrabing\data\othaim\booklet_'+str(i))
    page_images_booklet.to_csv(r'C:\Users\user\Desktop\web scrabing\data\othaim\booklet_'+str(i)+r'\booklet_'+str(i)+".csv", index=False)
    page_images_all=pd.concat([page_images_all,page_images_booklet])

page_images_all.to_csv(r'data\othaim\booklet_othaim.csv', index=False)
page_images_all.reset_index(inplace=True)
page_images_all.drop("index",inplace=True,axis=1)
page_images_all


in weekly_offers_links: 0
in numOfWebPages: 1
in weekly_offers_links: 1


IndexError: list index out of range

In [50]:
pd.set_option('display.max_colwidth', None)

page_images_all
# page_images_all.reset_index(inplace=True)
# page_images_all.drop("index",inplace=True,axis=1)
# page_images_all[]

,img_link,image_page_number,booklet_name,booklet_link
0,https://www.alsoouq.com//uploads/2022/07/5484706956.jpg,1.0,عروض العثيم اليوم 28 يوليو 2022 الموافق 29 ذو الحجة 1443 عروض بالجملة الاوفر & عروض الويكند,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d9%8a%d9%88%d9%85-28-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022-%d8%a7%d9%84%d9%85%d9%88%d8%a7%d9%81%d9%82-29-%d8%b0%d9%88-%d8%a7.html
1,https://www.alsoouq.com//uploads/2022/07/9820116987.jpg,1.0,عروض العثيم اليوم 28 يوليو 2022 الموافق 29 ذو الحجة 1443 عروض بالجملة الاوفر & عروض الويكند,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d9%8a%d9%88%d9%85-28-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022-%d8%a7%d9%84%d9%85%d9%88%d8%a7%d9%81%d9%82-29-%d8%b0%d9%88-%d8%a7.html
2,https://www.alsoouq.com//uploads/2022/07/4675580916.jpg,1.0,عروض العثيم اليوم 28 يوليو 2022 الموافق 29 ذو الحجة 1443 عروض بالجملة الاوفر & عروض الويكند,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d9%8a%d9%88%d9%85-28-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022-%d8%a7%d9%84%d9%85%d9%88%d8%a7%d9%81%d9%82-29-%d8%b0%d9%88-%d8%a7.html
3,https://www.alsoouq.com//uploads/2022/07/90469279.jpg,1.0,عروض العثيم اليوم 28 يوليو 2022 الموافق 29 ذو الحجة 1443 عروض بالجملة الاوفر & عروض الويكند,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d9%8a%d9%88%d9%85-28-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022-%d8%a7%d9%84%d9%85%d9%88%d8%a7%d9%81%d9%82-29-%d8%b0%d9%88-%d8%a7.html
4,https://www.alsoouq.com//uploads/2022/07/7077529347.jpg,1.0,عروض العثيم اليوم 28 يوليو 2022 الموافق 29 ذو الحجة 1443 عروض بالجملة الاوفر & عروض الويكند,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d9%8a%d9%88%d9%85-28-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022-%d8%a7%d9%84%d9%85%d9%88%d8%a7%d9%81%d9%82-29-%d8%b0%d9%88-%d8%a7.html
...,...,...,...,...
202,https://www.alsoouq.com//uploads/2022/07/7477848675.jpg,5.0,عروض العثيم الأسبوعية تسري حتى 19 يوليو 2022 الموافق 20 ذو الحجة 1443 عروض التذكير باليوم الأخير,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d8%a3%d8%b3%d8%a8%d9%88%d8%b9%d9%8a%d8%a9-%d8%aa%d8%b3%d8%b1%d9%8a-%d8%ad%d8%aa%d9%89-19-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022.html
203,https://www.alsoouq.com//uploads/2022/07/3835060402.jpg,6.0,عروض العثيم الأسبوعية تسري حتى 19 يوليو 2022 الموافق 20 ذو الحجة 1443 عروض التذكير باليوم الأخير,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d8%a3%d8%b3%d8%a8%d9%88%d8%b9%d9%8a%d8%a9-%d8%aa%d8%b3%d8%b1%d9%8a-%d8%ad%d8%aa%d9%89-19-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022.html
204,https://www.alsoouq.com//uploads/2022/07/9657984676.jpg,6.0,عروض العثيم الأسبوعية تسري حتى 19 يوليو 2022 الموافق 20 ذو الحجة 1443 عروض التذكير باليوم الأخير,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d8%a3%d8%b3%d8%a8%d9%88%d8%b9%d9%8a%d8%a9-%d8%aa%d8%b3%d8%b1%d9%8a-%d8%ad%d8%aa%d9%89-19-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022.html
205,https://www.alsoouq.com//uploads/2022/07/9404603447.jpg,6.0,عروض العثيم الأسبوعية تسري حتى 19 يوليو 2022 الموافق 20 ذو الحجة 1443 عروض التذكير باليوم الأخير,https://www.alsoouq.com/saudi-arabia-offers/offers-othaim/%d8%b9%d8%b1%d9%88%d8%b6-%d8%a7%d9%84%d8%b9%d8%ab%d9%8a%d9%85-%d8%a7%d9%84%d8%a3%d8%b3%d8%a8%d9%88%d8%b9%d9%8a%d8%a9-%d8%aa%d8%b3%d8%b1%d9%8a-%d8%ad%d8%aa%d9%89-19-%d9%8a%d9%88%d9%84%d9%8a%d9%88-2022.html


In [12]:

from itertools import count


def download_img(booklet):

	global data
	global count
	while True:
		try:
			img_url=data.pop()
			res = requests.get(img_url, stream=True)
			filename = f"data/othaim/booklet_{booklet}/{img_url.split('/')[-1]}"
			count= count+1
			page_n=count
			with open(filename, 'wb') as f:
				for block in res.iter_content(1024):
					f.write(block)
			print("page number: "+str(page_n)+" downloded")

		except:
			break

In [17]:
data= ''
count=-1
for booklet in range(9):
   
    print("booklet number: "+ str(booklet))
    data =deque(pd.read_csv(fr'data/othaim/booklet_{booklet}/booklet_{booklet}.csv').reset_index()["img_link"])
    threads=[]
    count=-1
    for i in range(3):
        thread = Thread(target=download_img,args=[booklet])
        thread.daemon = True
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()


booklet number: 7
page number: 0 downloded
page number: 1 downloded
page number: 2 downloded
page number: 3 downloded
page number: 4 downloded
page number: 5 downloded
page number: 6 downloded
page number: 7 downloded
page number: 8 downloded
page number: 9 downloded
page number: 10 downloded
page number: 11 downloded
page number: 12 downloded
page number: 13 downloded
page number: 14 downloded
page number: 15 downloded
page number: 16 downloded
page number: 17 downloded
page number: 18 downloded
page number: 19 downloded
page number: 20 downloded
page number: 21 downloded
page number: 22 downloded
page number: 23 downloded
page number: 24 downloded
page number: 25 downloded


In [75]:
dt = pd.DataFrame({"img_link":[],"image_page_number":[],"booklet_name":[],"booklet_link":[]})

for booklet in range(5):
  dt=pd.concat([dt,pd.read_csv(fr'data/pandah/booklet_{booklet}/booklet_{booklet}.csv')])
dt.to_csv(r'data\pandah\booklet_pandah.csv', index=False)

In [47]:
def addLinkCol(path):
    CSV=pd.read_csv(path) #rf'data/othaim/booklet_1/booklet_1.csv')
    t=pd.DataFrame({"img_name":CSV["img_link"]})
    dt=pd.concat([CSV,t["img_name"].apply(lambda a: a.split('/')[-1] , )],axis=1)
    dt.to_csv(path, index=False)

In [64]:
for i in [0,2,3,5,6,8,9]:
    addLinkCol(rf'data/danube/booklet_{i}/booklet_{i}.csv')

In [73]:
addLinkCol(rf'data/tamimi/booklet_tamimi.csv')

In [84]:
def remove(images,supermarket,booklet):
    bookletCSV=pd.read_csv(rf'data/{supermarket}/booklet_{booklet}/booklet_{booklet}.csv')
    suCSV= pd.read_csv(rf'data/{supermarket}/booklet_{supermarket}.csv')
    for  image in images:
        display(bookletCSV[bookletCSV["img_name"]==image])
        display(suCSV[suCSV["img_name"]==image])
        bookletCSV.drop(bookletCSV[bookletCSV["img_name"]==image].index,inplace=True)
        suCSV.drop(suCSV[suCSV["img_name"]==image].index,inplace=True)
        display(bookletCSV[bookletCSV["img_name"]==image])
        display(suCSV[suCSV["img_name"]==image])
        os.remove(rf'data/{supermarket}/booklet_{booklet}/{image}')
    bookletCSV.to_csv(rf'data/{supermarket}/booklet_{booklet}/booklet_{booklet}.csv', index=False)
    suCSV.to_csv(rf'data/{supermarket}/booklet_{supermarket}.csv', index=False)

In [85]:
remove(["1220932763.jpg"],"othaim","0")

,img_link,image_page_number,booklet_name,booklet_link,img_name,img_name.1
0,https://www.alsoouq.com//uploads/2022/07/12209...,1.0,عروض الجوالات من العثيم اليوم 29 يوليو 2022 ال...,https://www.alsoouq.com/saudi-arabia-electroni...,1220932763.jpg,1220932763.jpg


,img_link,image_page_number,booklet_name,booklet_link,img_name
0,https://www.alsoouq.com//uploads/2022/07/12209...,1.0,عروض الجوالات من العثيم اليوم 29 يوليو 2022 ال...,https://www.alsoouq.com/saudi-arabia-electroni...,1220932763.jpg


,img_link,image_page_number,booklet_name,booklet_link,img_name,img_name.1


,img_link,image_page_number,booklet_name,booklet_link,img_name


In [124]:
def redownload(images,supermarket,booklet):
    bookletCSV=pd.read_csv(rf'data/{supermarket}/booklet_{booklet}/booklet_{booklet}.csv')
    for image in images:
        os.remove(rf'data/{supermarket}/booklet_{booklet}/{image}')
        display(bookletCSV[bookletCSV["img_name"]==image])
        img_url=bookletCSV[bookletCSV["img_name"]==image].reset_index()["img_link"][0]
        res = requests.get(img_url, stream=True)
        filename = f"data/{supermarket}/booklet_{booklet}/{img_url.split('/')[-1]}"
        with open(filename, 'wb') as f:
            for block in res.iter_content(1024):
                f.write(block)


In [123]:
redownload({"1338112502.JPEG","3328346178.JPEG","4675580916.JPG","5484706956.JPG","6294712518.JPEG","7077529347.JPG","9377964170.JPEG","9820116987.JPG"},"othaim","1")

KeyError: 0